In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy import interpolate
import biosig as bs
import os
import _cut_signal as ct

def open_files(path):
    files = [file for file in os.listdir(path) if file[-4:] in ('.WFT', '.wft')]
    signals = pd.DataFrame({'time': [],
                        'incid':[],
                        'reflect':[]})

    for file in files:
        HDR=bs.header(path+file)
        sr = float(HDR.split('\n')[9][HDR.split('\n')[9].find(':')+2:-1])
        data=bs.data(path+file)


        res = []
        for item in data:
            res.append(item[0])
        if signals.incid.sum() == 0:
            signals.incid = res
        elif signals.incid.max() > max(res):
            signals.reflect = res
        else:
            signals.reflect = signals.incid
            signals.incid = res
    signals.time = np.arange(len(data))/sr

    return signals
    



In [5]:
way = '/Users/lubimyj/Git/experiments/Plansee-Tungstan-W_cover/'
materials = ['Denal920','DX2']
specimens = ['1','2','3','4','5','6']
to_IR_data = '/Specimen'
to_raw_data = '/test/Original WFT Files/'
IR_index = '_IR/'
subFold = '/IRCamera/Specimen'
testFold = 'test/EXP #1/'
dataFold = '/FLT/'

material = materials[1]
specimen = specimens[5]



In [6]:
mech_data_path = way + material + subFold + specimen + '/'
raw_data_path = way + material + subFold + specimen + to_raw_data
thermo_data_path = way + material + subFold + specimen + to_IR_data + specimen + IR_index
mech_data = pd.read_csv(mech_data_path+'result_data.txt', sep='\t')
#thermo_data = pd.read_csv(thermo_data_path +'Temperature_Spec_'+specimen+'.csv', sep = '\t' )           ##Treshhold by MIN
thermo_data = pd.read_csv(thermo_data_path +'Temperature_Spec_'+specimen+'Max_Tr.csv', sep = '\t' )     ##Treshhold by MAX
raw_data = open_files(raw_data_path)
title = 'Tr_0.9'




In [28]:
mech_data.columns

Index(['time', 'incid', 'trans', 'refl', 'U_in', 'U_out', 'eng_strain',
       'eng_stress', 'true_strain', 'true_stress', 'F_in', 'F_out'],
      dtype='object')

In [127]:
fig1 = go.Figure()
fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f'Comparison IncidentWave vs Temperature, {material} specimen {specimen} {title}')

fig1.add_trace(go.Scatter(x = thermo_data.time-107e-6, 
                            y = thermo_data[title], 
                            name = 'Temperature, C'),
                            secondary_y=True)
fig1.add_trace(go.Scatter(x = raw_data.time, 
                            y = raw_data.incid, 
                            name = 'Raw'))

fig1.update_xaxes(title_text="Time, s")

fig1.update_yaxes(title_text="Voltage", secondary_y=False)
fig1.update_yaxes(title_text="Temperature", secondary_y=True, range=[-thermo_data[title].max()-10,thermo_data[title].max()+10])

fig1.show()

In [130]:
fig1 = go.Figure()
len(fig1._data_objs)

0

In [8]:
#func_sign, func_time = cut_signal(mech_data.true_strain, thermo_data[title], mech_data.time, thermo_data.time)
#func_sign, func_time = cut_signal(thermo_data[title], thermo_data.time)
signal1 = mech_data.true_strain.values
signal2 = thermo_data[title].values
time1 = mech_data.time.values
time2 = thermo_data.time.values
#temperature = ct.cut_signal(mech_data.true_strain, thermo_data[title], mech_data.time, thermo_data.time)



In [65]:
def find_slope(sign1, sign2): #sign1 - Сигнал деформаций (опорный), sign2 - Сигнал температуры
    #Функция ищет восходящий фронт сигнала по порогу чувствительности 0.01 от максимальной величины сигнала
    tr1 = sign1.max()*0.1
    tr2 = sign2.max()*0.055

    return (np.where(sign1 > tr1)[0][0], 
            np.where(sign2 > tr2)[0][0])

def my_interpolation(t, data):
    xnew = np.arange(0,max(t),500*1e-9) #500 нс - величина обратная Частоте дискретизации 2ГГц
    f = interpolate.interp1d(t, data, kind = 'quadratic')
    ynew = f(xnew)
    return (xnew, ynew)

def cut_signal(signal1, signal2, time1, time2):
    #Функция отрезвет сигнал температуры в соответствии с длиной сигнала деформаций
    #сначала интерполируем второй синал чтобы его можно было двигать плавно
    newTime2, newSignal2 = my_interpolation(time2, signal2)   
    p1, p2 = find_slope(signal1, newSignal2)
    #Сдвигаем сигнал температуры влево так чтобы время обнаруженного подъема р1 совпало со временем р1
    newTime2 = newTime2 - newTime2[p2] + time1[p1]
    #new_signal2.time = new_signal2.time - 107e-6
    #отрезвем сигнал температуры сначала так чтобы были только положительные времена
    zero = np.where(newTime2 >= 0)[0][0]
    newTime2 = newTime2[zero:len(signal1)+zero]
    newSignal2 = newSignal2[zero:len(signal1)+zero]
    
    
    return (newTime2, newSignal2)
newTime, temperature = cut_signal(signal1, signal2, time1, time2)

In [120]:
plots = [
            go.Scatter(name=r'$Incident$', y=raw_data.incid, x=raw_data.time,
                       mode='lines', visible=True, yaxis = 'y'),

            go.Scatter(name=r'$Reflected$', y=raw_data.reflect, x=raw_data.time,
                       mode='lines', visible=True, yaxis = 'y'),
            
            go.Scatter(name=r'$Incident$', y=mech_data.incid, x=mech_data.time,
                       mode='lines', visible=False),
            go.Scatter(name=r'$Reflected$', y=mech_data.refl, x=mech_data.time,
                       mode='lines', visible=False),

            go.Scatter(name=r'$U_{in}$', y=mech_data.U_in, x=mech_data.time,
                       mode='lines', visible=False),

            go.Scatter(name=r'$F_{in}$', y=mech_data.F_in, x=mech_data.time,
                       mode='lines', visible=False),

            go.Scatter(name=r'$V_{in}$', y=mech_data.U_out, x=mech_data.time,
                       mode='lines', visible=False),

            go.Scatter(name=r'$\sigma_{Engineering}$', y=mech_data.eng_stress,
                       x=mech_data.eng_strain, mode='lines', visible=False),

            go.Scatter(name=r'$True$', y=mech_data.true_stress,
                       x=mech_data.true_strain, mode='lines', visible=False),

            go.Scatter(name=r'$True$', y=temperature,
                       x=mech_data.true_strain, mode='lines', visible=False, yaxis = 'y2')
        ]
buttons = [
            dict(label='Raw Signals',
                 method='update',
                 args=[{'visible': [True] * 2 + [False] * 8},
                       {'xaxis': {'title': 'Time [μs]'},
                        'yaxis': {'title': 'Amplitude [V]'},
                        'showlegend': True}]),

            dict(label='Corrected Signals',
                 method='update',
                 args=[{'visible': [False]* 2 + [True] * 2 + [False] * 6},
                       {'xaxis': {'title': 'Time [μs]'},
                        'yaxis': {'title': 'Amplitude [V]'},
                        'showlegend': True}]),

            dict(label='Displacement',
                 method='update',
                 args=[{'visible': [False] * 4 + [True] + [False] * 5},
                       {'xaxis': {'title': 'Time [μs]'},
                        'yaxis': {'title': 'Displacement [m]'},
                        'showlegend': True}]),

            dict(label='Forces',
                 method='update',
                 args=[{'visible': [False] * 5 + [True] + [False] * 4},
                       {'xaxis': {'title': 'Time [μs]'},
                        'yaxis': {'title': 'Force [N]'},
                        'showlegend': True}]),

            dict(label='Velocities',
                 method='update',
                 args=[{'visible': [False] * 6 + [True] + [False] * 3},
                       {'xaxis': {'title': 'Time [μs]'},
                        'yaxis': {'title': 'Velocity [m/s]'},
                        'showlegend': True}]),

            dict(label='Stress - Strain',
                 method='update',
                 args=[{'visible': [False] * 7 + [True] * 2 + [False]},
                       {'xaxis': {'title': 'Strain'},
                        'yaxis': {'title': 'Stress [MPa]'},
                        'showlegend': True}]),
            dict(label='Temperature',
                 method='update',
                 args=[{'visible': [False] * 8 + [True] * 2},
                       {'xaxis': {'title': 'Strain'},
                        'yaxis': {'title': 'Stress, [MPa]'},
                        'showlegend': True}]),
                  
                        ]   
layot = go.Layout(title_x=0.5, template='none',
                    title = 'Specimen',
                    width=800, height=600,
                      font=dict(size=16),
                      legend=dict(yanchor='top', xanchor='left', y=1, x=1.15),
                      margin=dict(l=100, r=100, t=100, b=100),
                      updatemenus=[dict(active=0, buttons=buttons, xanchor='left', x=0, y=1.15)],
                      yaxis = dict(),
                      yaxis2 = dict(side = 'right', overlaying="y")
                      )

fig = go.Figure(data = plots, layout = layot)


fig.show()

In [26]:
#fig1 = go.Figure()
fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.update_layout(template = 'plotly_white', width=800, height=600,
                  title = 'Comparison Strain vs Temperature')
fig1.add_trace(go.Scatter(x = mech_data.time, 
                            y = mech_data.true_strain,
                            name = 'True Strain'),
                            secondary_y=False)
fig1.add_trace(go.Scatter(x = mech_data.time, 
                            y = mech_data.true_stress,
                            name = 'True Stress'),
                            secondary_y=False)
fig1.add_trace(go.Scatter(x = newTime, 
                            y = temperature, 
                            name = 'Temperature'),
                            secondary_y=True)

fig1.update_xaxes(title_text="Time, s")

fig1.update_yaxes(title_text="Strain", secondary_y=False)
fig1.update_yaxes(title_text="Temperature", secondary_y=True)

fig1
fig1.show()

In [18]:
fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f'True Strain vs Temperature, {material} specimen {specimen}')
fig2.add_trace(go.Scatter(x = mech_data.true_strain, 
                            y = mech_data.true_stress,
                            name = 'True Stress'),
                            secondary_y=False)
fig2.add_trace(go.Scatter(x = mech_data.true_strain, 
                            y = temperature.value, 
                            name = 'Temperature'),
                            secondary_y=True)

fig2.update_xaxes(title_text="Strain")

fig2.update_yaxes(title_text="Stress, MPa", secondary_y=False)
fig2.update_yaxes(title_text="Temperature, C", secondary_y=True)

fig2
fig2.show()

In [19]:
fig3 = go.Figure()
fig3.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f'True Strain vs Temperature, {material} specimen {specimen}')
fig3.add_trace(go.Scatter(x = mech_data.true_strain, 
                            y = temperature.value/mech_data.true_stress, 
                            name = 'Temperature'))

fig3.update_xaxes(title_text="Strain")

fig3.update_yaxes(title_text="Betha")

fig3
fig3.show()